# Aufgabe 1: Markup - server part
Dieses File dient nur zur kurzen Beschreibung des Codes!!!
Die Ausführung erfolgt über auskunft.ipynb und schnittstelle.ipynb und dem script auskunft_schnittstelle.py

In [1]:
import logging
import socket
import constCS
from context import lab_logging

lab_logging.setup(stream_level=logging.INFO)  # init loging channels for the lab

Bereitstellen der Packages für Socket und Log.

In [2]:
class Auskunft:
    _logger = logging.getLogger("vs2lab.lab1.auskunft_schnittstelle.Auskunft")
    _serving = True
    telebook = {'Tim':1111, 'Stefan':2222, 'Lisa':3333}

Klasse anlegen, Logger erstellen und Telefonbuch anlegen.

In [3]:
    def __init__(self):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.bind((constCS.HOST, constCS.PORT))
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # prevents errors due to "addresses in use"
        self.sock.settimeout(3)  # time out in order not to block forever
        self._logger.info("\nServer bound to socket " + str(self.sock) + "\n")


Socket erstellen und an entsprechenden Port binden. Port und Host findet sich in der externen Datei constCS.py wieder.
Log-Eintrag für die Bindung eintragen.

In [4]:
    def serve(self):
        self.sock.listen(1)
        while self._serving:  # as long as _serving (checked after connections or socket timeouts)
            try:
                (connection, address) = self.sock.accept()  # returns new socket and address of client
                while True:  # forever
                    data = connection.recv(1024)
                    data = data.decode('ascii')  # receive data from client
                    self._logger.info("\nMessage " + data + " recieved from Client\n")
                    
                    if not data:
                        break  # stop if client stopped
                    elif data == "getall":
                        data = str(self.telebook)
                    elif data in self.telebook:
                        data = str(self.telebook.get(data))
                    else:
                        data = "No entry found!"

                    connection.send(data.encode('ascii'))  # return sent data plus an "*"
                    self._logger.info("\nServer sends " + data + " over Socket" + str(self.sock) + "\n")
                    
                connection.close()  # close the connection
                self._logger.info("\nServer socket closed!\n")
            except socket.timeout:
                pass  # ignore timeouts
        self.sock.close()
        self._logger.info("\nServer down.\n")


1)
Über self-sock.listen() lauscht der Server auf dem Socket. Bei eingehender Nachricht wird diese akzeptiert und decodiert. Anschließend geschieht ein Log-Eintrag.

2)
Die Auswertung der Nachricht erfolgt über if-Schleifen
- getall gibt das gesamte Telefonbuch zurück 
- get gibt jeweil nur die enstprechende Nummer der Person zurück oder einen nicht vorhandenen Eintrag.
Das Ergebniss wird in data geschrieben.

3)
connectinon.send() wird das Ergebnis encodiert und an den Client zurückgeschickt. Anschließend geschieht wieder ein Log-Eintrag

4)
Server Socket schließen und letzten Log-.Eintrag anlegen

Finished